# Proyecto 2

### En la primera parte, vamos a exportar la paquetería que se necesitá para la resolución del problema, para esto, primero se va a revisar que los datos en el archivo estén todos correctos. Vamos a leerlos y, posteriormente, vamos a identificar posibles problemas en estos

In [12]:
# Used to read the data file
import pandas as pd

# Used in the definition of the model and its initial condition
import numpy as np

# Used for plots other than those generated by the parameter estimation module
import matplotlib.pyplot as plt
%matplotlib inline
import PDEparams as pde

In [13]:
df = pd.read_csv('CoV2019.csv')

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 8 columns):
Date of report    41 non-null object
Days              41 non-null int64
Total             41 non-null int64
China             41 non-null int64
Death China       41 non-null int64
Other             41 non-null int64
Death Outside     41 non-null int64
Death Globally    41 non-null int64
dtypes: int64(7), object(1)
memory usage: 2.7+ KB


In [15]:
df.head()

,Date of report,Days,Total,China,Death China,Other,Death Outside,Death Globally
0,01/20/20,21,282,278,6,4,0,6
1,01/21/20,22,346,341,6,4,0,6
2,01/23/20,24,581,571,17,7,0,17
3,01/24/20,25,846,830,25,11,0,25
4,01/25/20,26,1320,1297,41,23,0,41


Los archivos en primera instancia se ven bastante bien, por lo cual, ahora se va a utilizar el módelo ISR para este sistema, el cual viene dada por:

$$\begin{align}
\frac{\mathrm{d} S}{\mathrm{d} t} &= -\beta\, S\, I\,\\
\frac{\mathrm{d} I}{\mathrm{d} t} &= +\beta\,S\,I-\gamma\,I\\
\frac{\mathrm{d} R}{\mathrm{d} t} &= \gamma I
\end{align}$$

donde <span style="color:red">$S$</span> es la población susceptible, <span style="color:red">$I$</span> es la población infectada y <span style="color:red">$R$</span> es la población inmune o fallecida.

En los datos, se puede observar que vienen los features 'china' y 'death china', por lo que, se asumirá que el feature de 'China' corresponde a los enfermos por Covid-19 en China, i, e., <span style="color:red">$I$</span>, el feature de 'Death China' los muertos por Covid en China y que no están considerados ya en los infectados, además, dado que no se sabe quienes son inmunes o no hay conocimiento de ello aún, se tomará como <span style="color:red">$R$</span> solo a los muertos por Covid en China y dado que la población de china al inicio del año era de 1 415 368 746 y no se encontró la evolución de la población de china durante ese periodo, i. e., no se encontró la información respecto a los nacimientos que hubo en china ni los fallecimientos por otras enfermedades en el periodo 20/01/2020-01/03/2020, se asumirá que la población en china en esa temporalidad, es la inicial menos los muertos reportados por Covid en ese día, por lo cual, la población susceptible <span style="color:red">$S$</span> corresponde a 1 415 368 746 menos los features 'Death China' y 'China'.

In [ ]:
df.rename(columns={'China':'I',

In [11]:
def ISR(z,t,b,g):
    S,I,R=z
    return [-b*S*I, b*S*I-g*I, g*I]

In [10]:
df['Days'].values.min()

21